# Schwarzschild Black Hole Ray Tracer

This notebook implements a physically accurate ray tracer for visualizing a **Schwarzschild black hole** (non-rotating) using geodesic equations from General Relativity.

## Physics Overview

The simulation solves the **geodesic equations** for photons (light rays) in curved spacetime:

$$\frac{d^2\phi}{d\lambda^2} = -\frac{2}{r}\frac{dr}{d\lambda}\frac{d\phi}{d\lambda}$$

$$\frac{d^2r}{d\lambda^2} = r\left(\frac{d\phi}{d\lambda}\right)^2\left(1 - \frac{3GM}{2rc^2}\right)$$

Where:
- $\lambda$ = affine parameter (path length along light ray)
- $r$ = radial coordinate
- $\phi$ = azimuthal angle
- $Rs = 2GM/c^2$ = Schwarzschild radius (event horizon)

## Key Physical Features

- **Event Horizon**: $r = Rs$ - Nothing escapes from inside
- **Photon Sphere**: $r = 1.5Rs$ - Unstable orbit where light circles the black hole
- **ISCO**: $r = 3Rs$ - Innermost Stable Circular Orbit (inner edge of accretion disk)
- **Gravitational Lensing**: Light rays bend according to spacetime curvature

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import time

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

## 2. Physical Constants and Configuration

We use **geometrized units** where $G = c = M = 1$, so all distances are measured in units of the Schwarzschild radius $Rs$.

In [ ]:
# Simulation Parameters
WIDTH = 800         # Image width (pixels)
HEIGHT = 600        # Image height (pixels)
FOV = 60.0          # Field of view (degrees)

# Integration Parameters
STEP_SIZE = 0.05    # RK4 integration step size
MAX_STEPS = 5000    # Maximum steps before assuming ray escaped

# Physical Radii (in units of Rs)
RS = 1.0                # Schwarzschild Radius (Event Horizon)
R_PHOTON = 1.5 * RS     # Photon sphere radius
R_ISCO = 3.0 * RS       # Innermost Stable Circular Orbit
R_DISK_MAX = 12.0 * RS  # Outer edge of accretion disk

# Camera Setup
CAM_DIST = 30.0     # Distance from black hole
CAM_PITCH = 25.0    # Viewing angle above disk plane (degrees)

print(f"Configuration:")
print(f"  Resolution: {WIDTH}×{HEIGHT}")
print(f"  Camera distance: {CAM_DIST} Rs")
print(f"  Event horizon: {RS} Rs")
print(f"  Photon sphere: {R_PHOTON} Rs")
print(f"  ISCO (disk inner edge): {R_ISCO} Rs")

## 3. Geodesic Equations (Numba-Accelerated)

These functions implement the core physics using **4th-order Runge-Kutta** integration.

In [ ]:
@njit
def get_derivatives(state):
    """
    Calculate derivatives for Schwarzschild geodesic equations.
    
    Args:
        state: [r, r_dot, phi, phi_dot]
    
    Returns:
        [r_dot, r_ddot, phi_dot, phi_ddot]
    """
    r = state[0]
    r_dot = state[1]
    phi = state[2]
    phi_dot = state[3]

    # Geodesic equations for photons in Schwarzschild metric
    phi_ddot = -2.0 / r * r_dot * phi_dot
    r_ddot = r * (phi_dot ** 2) * (1.0 - (1.5 * RS) / r)

    return np.array([r_dot, r_ddot, phi_dot, phi_ddot])

@njit
def rk4_step(state, h):
    """
    Perform a single 4th-order Runge-Kutta integration step.
    
    Args:
        state: Current state vector
        h: Step size
    
    Returns:
        Updated state vector
    """
    k1 = get_derivatives(state)
    k2 = get_derivatives(state + 0.5 * h * k1)
    k3 = get_derivatives(state + 0.5 * h * k2)
    k4 = get_derivatives(state + h * k3)

    return state + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

print("Geodesic integration functions compiled successfully!")

## 4. Accretion Disk Texture

The accretion disk includes:
- **Radial temperature gradient**: Hotter (yellow) near ISCO, cooler (red) at outer edge
- **Doppler beaming**: Approaching side appears brighter
- **Procedural rings**: Simulates turbulent structure

In [ ]:
@njit
def texture_lookup(r, phi, x_hit, y_hit, z_hit):
    """
    Generate procedural color for the accretion disk.
    
    Args:
        r: Radial distance where ray hit disk
        phi: Angular coordinate (unused here)
        x_hit, y_hit, z_hit: 3D hit coordinates
    
    Returns:
        RGB color array [0-1]
    """
    # Procedural ring pattern (smooth concentric rings)
    ring_noise = 0.6 * np.sin(r * 2.0) + 0.4 * np.sin(r * 5.0)
    brightness = 0.6 + 0.4 * ring_noise

    # Doppler beaming: approaching side (x < 0) is brighter
    # Assumes disk rotates counter-clockwise
    doppler = 1.0 + 0.5 * (x_hit / r)

    # Temperature gradient: hotter near center
    dist_norm = (r - R_ISCO) / (R_DISK_MAX - R_ISCO)
    dist_norm = max(0.0, min(1.0, dist_norm))

    # Color mapping: Orange→Yellow near center, Red at edges
    col_r = 1.0 * brightness * doppler
    col_g = (0.8 - 0.6 * dist_norm) * brightness * doppler
    col_b = (0.3 - 0.3 * dist_norm) * brightness * doppler

    return np.array([col_r, col_g, col_b])

print("Texture function compiled!")

## 5. Ray Marching (Ray Tracing Engine)

This function traces a single light ray backward from the camera through curved spacetime.

In [ ]:
@njit
def ray_march(ray_origin, ray_dir):
    """
    Trace a single ray through curved spacetime.
    
    Args:
        ray_origin: Starting position (camera)
        ray_dir: Initial ray direction
    
    Returns:
        RGB color [0-1] of what the ray hits
    """
    # 1. Setup: Transform to planar coordinates
    # Photon trajectories in Schwarzschild are planar
    pos = ray_origin
    vel = ray_dir

    # Angular momentum vector L = r × v
    L = np.cross(pos, vel)
    L_norm = np.linalg.norm(L)

    # If L ≈ 0, ray is radial (straight into BH)
    if L_norm < 1e-6:
        return np.array([0.0, 0.0, 0.0])

    # Define orthonormal basis for plane of motion
    r_hat = pos / np.linalg.norm(pos)
    n_hat = L / L_norm  # Normal to plane
    phi_hat = np.cross(n_hat, r_hat)

    # Initial state in polar coordinates
    r_init = np.linalg.norm(pos)
    phi_init = 0.0
    dr_init = np.dot(vel, r_hat)
    dphi_init = np.dot(vel, phi_hat) / r_init

    state = np.array([r_init, dr_init, phi_init, dphi_init])

    # 2. Integration loop
    for i in range(MAX_STEPS):
        old_r = state[0]
        old_phi = state[2]

        # Integrate one step
        state = rk4_step(state, STEP_SIZE)

        new_r = state[0]
        new_phi = state[2]

        # Check 1: Event horizon
        if new_r < RS:
            return np.array([0.0, 0.0, 0.0])  # Black

        # Check 2: Escaped to infinity
        if new_r > 50.0:
            return np.array([0.02, 0.02, 0.05])  # Dark blue space

        # Check 3: Accretion disk intersection
        # Reconstruct 3D position
        pos_3d_new = new_r * (np.cos(new_phi) * r_hat + np.sin(new_phi) * phi_hat)
        pos_3d_old = old_r * (np.cos(old_phi) * r_hat + np.sin(old_phi) * phi_hat)

        # Check if crossed Y=0 plane (disk is at Y=0)
        y0 = pos_3d_old[1]
        y1 = pos_3d_new[1]

        if (y0 > 0 and y1 < 0) or (y0 < 0 and y1 > 0):
            # Crossed the plane! Find exact intersection
            fraction = abs(y0) / (abs(y0) + abs(y1))
            hit_r = old_r + (new_r - old_r) * fraction

            # Check if within disk bounds
            if R_ISCO < hit_r < R_DISK_MAX:
                hit_pos = pos_3d_old + (pos_3d_new - pos_3d_old) * fraction
                return texture_lookup(hit_r, 0, hit_pos[0], hit_pos[1], hit_pos[2])

    return np.array([0.0, 0.0, 0.0])

print("Ray marching function compiled!")

## 6. Main Rendering Function

Renders the full image by tracing one ray per pixel, parallelized across CPU cores.

In [ ]:
@njit(parallel=True)
def render_image(width, height, cam_pos, cam_target, fov):
    """
    Render a black hole image by ray tracing.
    
    Args:
        width, height: Image resolution
        cam_pos: Camera 3D position
        cam_target: Point camera looks at (usually [0,0,0])
        fov: Field of view in degrees
    
    Returns:
        RGB image array [height, width, 3]
    """
    image = np.zeros((height, width, 3))
    aspect_ratio = width / height
    fov_rad = np.radians(fov)
    half_height = np.tan(fov_rad / 2.0)
    half_width = aspect_ratio * half_height

    # Camera coordinate system
    w = (cam_target - cam_pos)
    w = w / np.linalg.norm(w)  # Forward
    up = np.array([0.0, 1.0, 0.0])
    u = np.cross(w, up)  # Right
    u = u / np.linalg.norm(u)
    v = np.cross(u, w)  # Up

    # Trace rays (parallelized)
    for y in prange(height):
        for x in prange(width):
            # Normalized device coordinates
            ndc_x = (x + 0.5) / width * 2.0 - 1.0
            ndc_y = 1.0 - (y + 0.5) / height * 2.0

            # Ray direction
            pixel_screen_pos = cam_pos + w + u * (ndc_x * half_width) + v * (ndc_y * half_height)
            ray_dir = pixel_screen_pos - cam_pos
            ray_dir = ray_dir / np.linalg.norm(ray_dir)

            color = ray_march(cam_pos, ray_dir)
            image[y, x] = color

    return image

print("Rendering function compiled!")

## 7. Render a Single Image

Let's create our first black hole visualization!

In [ ]:
# Setup camera position
pitch_rad = np.radians(CAM_PITCH)
cam_y = CAM_DIST * np.sin(pitch_rad)
cam_z = -CAM_DIST * np.cos(pitch_rad)
cam_pos = np.array([0.0, cam_y, cam_z])
cam_target = np.array([0.0, 0.0, 0.0])

print("Compiling JIT kernels (first run only, ~5-10 seconds)...")
start_time = time.time()
# Trigger compilation with small test
_ = render_image(10, 10, cam_pos, cam_target, FOV)
print(f"Compilation done in {time.time() - start_time:.2f}s\n")

print(f"Rendering {WIDTH}×{HEIGHT} image...")
start_time = time.time()
img_data = render_image(WIDTH, HEIGHT, cam_pos, cam_target, FOV)
render_time = time.time() - start_time
print(f"Render complete in {render_time:.2f}s")
print(f"Performance: {(WIDTH * HEIGHT) / render_time / 1e6:.2f} Mrays/sec")

## 8. Display the Result

In [ ]:
plt.figure(figsize=(12, 9), dpi=100)
plt.imshow(np.clip(img_data, 0, 1))
plt.axis('off')
plt.title("Schwarzschild Black Hole\n(Ray traced using General Relativity)", 
          fontsize=14, pad=20)
plt.tight_layout()
plt.savefig('black_hole.jpg', bbox_inches='tight', dpi=150)
print("Image saved as 'black_hole.jpg'")
plt.show()

## 9. Create Animation (Optional)

Generate a video showing an orbital flyby around the black hole.

In [ ]:
import cv2
import os

# Animation settings
ANIM_WIDTH = 640
ANIM_HEIGHT = 480
NUM_FRAMES = 120
FPS = 24
OUTPUT_FILE = 'black_hole_orbit.mp4'

print(f"Generating {NUM_FRAMES}-frame animation...")
print(f"Resolution: {ANIM_WIDTH}×{ANIM_HEIGHT} @ {FPS}fps")
print(f"Estimated time: ~{NUM_FRAMES * 5 / 60:.1f} minutes\n")

frames = []
target = np.array([0.0, 0.0, 0.0])

for i in range(NUM_FRAMES):
    # Polar orbit trajectory
    theta = (2.0 * np.pi * i) / NUM_FRAMES
    cam_pos_i = np.array([0.0,
                          CAM_DIST * np.sin(theta),
                          -CAM_DIST * np.cos(theta)])

    # Render frame
    img_data = render_image(ANIM_WIDTH, ANIM_HEIGHT, cam_pos_i, target, FOV)
    img_uint8 = (np.clip(img_data, 0, 1) * 255).astype(np.uint8)
    frame_bgr = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2BGR)
    frames.append(frame_bgr)

    if (i + 1) % 10 == 0:
        print(f"  Frame {i + 1}/{NUM_FRAMES} complete")

# Save video
print(f"\nWriting video to {OUTPUT_FILE}...")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_FILE, fourcc, FPS, (ANIM_WIDTH, ANIM_HEIGHT))
for frame in frames:
    out.write(frame)
out.release()

if os.path.exists(OUTPUT_FILE):
    print(f"✓ Video saved: {os.path.abspath(OUTPUT_FILE)}")
else:
    print("✗ Error: Video was not created")

## Summary

This notebook demonstrated:

1. ✅ **Physical accuracy**: Real geodesic equations from General Relativity
2. ✅ **Numerical method**: 4th-order Runge-Kutta integration
3. ✅ **Performance**: Numba JIT compilation for near-GPU speeds on CPU
4. ✅ **Visualization**: Gravitational lensing, photon sphere, accretion disk
5. ✅ **Animation**: Orbital camera paths

### Next Steps

- Try different camera angles (change `CAM_PITCH`)
- Increase resolution for higher quality
- Experiment with disk parameters
- See `KerrBlackHole.ipynb` for rotating black holes!

### References

- [YouTube Tutorial](https://www.youtube.com/watch?v=8-B6ryuBkCM) - Original implementation reference
- Chandrasekhar (1983) - Mathematical Theory of Black Holes
- Luminet (1979) - First computer-generated black hole image